In [1]:
import ctypes
import weakref

In [2]:
def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [3]:
class Person:
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return f"Person(name={self.name})"

In [4]:
p1 = Person("Guido")
p2 = p1

we have two symbols that are pointing to the same object

In [5]:
p1 is p2

True

In [6]:
ref_count(id(p1))

2

In [7]:
del p2

In [8]:
ref_count(id(p1))

1

In [9]:
p1_id = id(p1)

In [10]:
ref_count(p1_id)

1

In [11]:
del p1 # object will be garbage collected 

In [12]:
ref_count(p1_id)
# there is no object at this address anymore.
# we will start to see some strange results

605844000

#### there is another type of reference we can hold to an object, that does not affect reference count

In [13]:
p1 = Person("Guido")
p1_id = id(p1)

In [14]:
ref_count(p1_id)

1

In [15]:
p2 = p1
ref_count(p1_id)

2

In [16]:
weak1 = weakref.ref(p1)
ref_count(p1_id)

2

In [17]:
weak1

<weakref at 0x0000021811FCC548; to 'Person' at 0x0000021811FC8248>

In [18]:
weak1 is p1

False

In [19]:
weak1() is p1 
# callable. the result of calling will give us the object weak1 is pointing to

True

In [20]:
print(weak1()) 

Person(name=Guido)


In [21]:
ref_count(p1_id)

2

In [22]:
p3 = weak1() # p3 is now a strong reference
ref_count(p1_id)

3

In [23]:
del p3
ref_count(p1_id)

2

In [24]:
del p2
ref_count(p1_id)

1

In [25]:
print(weak1()) 

Person(name=Guido)


In [26]:
weak1

<weakref at 0x0000021811FCC548; to 'Person' at 0x0000021811FC8248>

In [27]:
del p1


In [28]:
ref_count(p1_id) # we have deleted all strong references, this number is meaningless

1

In [29]:
weak1

<weakref at 0x0000021811FCC548; dead>

In [30]:
result = weak1()
print(result)

None


#### many of the built-in types do not support weak references

In [31]:
l = [1,2,3]
try:
    w = weakref.ref(l)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'list' object


## weak references against memory leaks

#### special dictionary

In [32]:
p1 = Person("Guido")

In [33]:
d = weakref.WeakKeyDictionary()

In [34]:
ref_count(id(p1))

1

if we store p1 in normal dictionary we will up the reference count

In [35]:
n = {p1:"Guido"}

In [36]:
ref_count(id(p1))

2

In [37]:
del n

In [38]:
ref_count(id(p1))

1

weakkey dict

In [39]:
d[p1] = "Guido"

In [40]:
ref_count(id(p1))

1

count weak references

In [41]:
weakref.getweakrefcount(p1)

1

In [42]:
d2 = weakref.WeakKeyDictionary()
d2[p1] = "Guido"

In [43]:
ref_count(id(p1)), weakref.getweakrefcount(p1)

(1, 2)

weak references are stored in instance dict (so we will have difficulties if class use slots)

In [44]:
p1.__weakref__ # linked list. 
# python doesn't provide us with any tools to iterate through this list

<weakref at 0x0000021811FE3C78; to 'Person' at 0x0000021811FC2A48>

how to see weak references contained in d dict?

In [45]:
hex(id(p1))

'0x21811fc2a48'

In [46]:
list(d.keyrefs())

[<weakref at 0x0000021811FE3908; to 'Person' at 0x0000021811FC2A48>]

#### once the weak reference is dead, it is automatically removed from WeakKeyDictionary

In [47]:
del p1

In [48]:
list(d.keyrefs())

[]

Object has to be hashable to use in WeakKeyDictionary

In [50]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def __eq__(self, other):
        return isinstance(other, Person) and self.name == other.name

In [51]:
p1 = Person("Guido")
p2 = Person("Guido")

In [52]:
hash(p1)

TypeError: unhashable type: 'Person'

In [53]:
d[p1] = "test"

TypeError: unhashable type: 'Person'

we need to implement the hash to fix it